這份 Notebook 示範 Dynamic Few shot prompting 技巧，題目是: 書籍分類

我們透過 embeddings 找到最相似的範例當作 few-shot, 效果應會比 colab 206 中使用 random few-shot 更優秀。


In [ ]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [14]:
import requests
import json
from pprint import pp

In [15]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')


In [16]:
def get_completion(messages, model="gpt-4.1-mini", temperature=0, max_completion_tokens=4000, logit_bias=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_completion_tokens": max_completion_tokens }
  if logit_bias:
    payload["logit_bias"] = logit_bias

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

用來分類的話，可以用更好的 embedding 模型

In [17]:
def get_embeddings(input, dimensions = 3072, model="text-embedding-3-large"):
  payload = { "input": input, "model": model, "dimensions": dimensions }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/embeddings', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["data"][0]["embedding"]
  else :
    return obj["error"]

# 準備向量資料庫

裡面會放 few-shot 要用的參考標準範例

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.7 MB/s eta 0:00:00


In [19]:
import chromadb
chroma_client = chromadb.Client()

collection = chroma_client.create_collection(name="collection5")

In [24]:
import pandas as pd
df = pd.read_csv("books-dataset-107.csv")

df

,title,description,category
0,Python 程式設計與實例應用 2/e,本書簡介\r\n 《Python程式設計與實例應用》是一本Python程式教學指南，適合所...,程式語言
1,選擇權商品模型化導論：使用 Python 語言 (附光碟),⊙以Python解決數學概念問題，掌握衍生性商品（如選擇權商品）模型化。\r\n⊙理論與實作...,程式語言
2,從零開始學 Python 程式設計 (第三版修訂版)(適用 Python 3.10 以上)(...,\r\n\tPython入門經典好書，暢銷回饋中！\r\n\t\r\n\t【本書特色】\r\...,程式語言
3,人工智慧 Python 程式設計,張教授為國立聯合大學客家研究學院文化創意與數位行銷學系的教授，長期投身於AI與數位工具的教學...,程式語言
4,Python + ChatGPT 零基礎 + 高效率學程式設計與運算思維 4/e,\r\n\r\nPython + ChatGPT\r\n\r\n零基礎 + 高效率\r\n...,程式語言
...,...,...,...
102,電腦軟體設計丙級技能檢定學術科｜使用 C#,依據勞動部勞動力發展署技能檢定中心最新學術科試題規劃解題\r\n\r\n術科篇\r\n＊最新...,考試認證
103,讀一遍就記得的唐詩課,一堂點閱破百萬的唐詩課視頻\r\n學生譽為「夢想中的國文老師」\r\n十二種人生意境主題，全...,其他
104,家傳,十二年之前，\r\n\r\n一套四冊的《傳家》登場，\r\n\r\n為華人父母打造給下一代的...,其他
105,圖解 Java 物件導向程式語言,●簡潔精要，易於快速理解與掌握程式技巧\r\n●兼顧運算思維與實用技巧，提高學習效果\r\n...,程式語言


將參考資料放入向量資訊庫:

In [25]:
for index, row in df.iterrows():
  # 這是要索引的內容
  index_text = f"""
  書名: ```{row["title"]}```
  描述: ```{str(row["description"])[0:50]}```
  """

  # 這是對應的文字內容
  doc_text = f"""
  書名: ```{row["title"]}```
  描述: ```{row["description"]}```
  分類: {str(row["category"])[0:50]}
  """

  collection.add(
    documents = doc_text,
    embeddings = get_embeddings(index_text),
    ids=f"book-{index}"
  )

## Dynamic(kNN) few-shot prompting

載入要分類的書籍資料

In [26]:
df2 = pd.read_csv("books-dataset-33.csv")

df2

,title,description,category
0,絕對會 Python 用場! 驚人的程式妙用,✨ 想不到Python還可以這麼玩!? ✨\r\n\r\n✨ 用天馬行空的範例 讓你陷入Py...,程式語言
1,AI世代必備！Python × ChatGPT 高效率工作術：從網路爬蟲到辦公室自動化超實務,最全面的 ChatGPT × Python 應用手冊！\r\n\r\n \r\n\r\nAI...,程式語言
2,Python 風格徹底研究｜超詳實、好理解的 Python 必學主題 (Dead Simpl...,多位Python官方社群的大神技術審校和推薦\r\n教您寫出Python風格的專業程式碼\r...,程式語言
3,Python 大數據專案 X 工程 X 產品 資料工程師的升級攻略 2/e,★☆★☆★ 獨家解析知名大數據專案，FinMind，帶你一窺大數據產品的發展過程，打造專屬個...,程式語言
4,AI 時代的管理數學：使用 R語言實作,如果你主要關注統計分析、數據可視化、線性代數、初等微積分，\r\n\r\n \r\n\r\n...,人工智慧
5,史上最強 Python 入門邁向頂尖高手之路王者歸來 3/e (全彩印刷),天瓏購書獨家贈送習題解答\r\n\r\n史上最強\r\n\r\nPython入門\r\n\r...,程式語言
6,AI世代：從政治哲學反思人工智慧的衝擊,人臉辨識、數位威權、同溫層效應……\r\n科技是中立的嗎？\r\n科技真的能帶來更好的未來嗎...,人工智慧
7,電子商務與 ChatGPT：物聯網 ‧ KOL直播 ‧ 區塊鏈‧ 社群行銷 ‧ 大數據 ‧...,\r\n\t電子商務是利用網路進行銷售或交換產品與服務，並達到降低成本要求的現代化經營模式，...,物聯網 IoT
8,權力與進步：科技變革與共享繁榮之間的千年辯證,全球獨家中文版！《金融時報》年度選書\r\n\r\n關心AI時代人類命運的必讀之作\r\n\...,人工智慧
9,電腦軟體設計丙級技能檢定學術科｜適用 C++ (第三版),依據勞動部勞動力發展署技能檢定中心最新啟用之學術科試題規劃解題\r\n\r\n術科篇\r\n...,考試認證


逐筆跑分類預測

In [28]:
for index, row in df2.iterrows():
  # 找最相似的10筆範例
  query_text = f"""
  書名: ```{row["title"]}```
  描述: ```{row["description"]}```
  """

  results = collection.query(
    query_embeddings = get_embeddings(query_text),
    n_results=10
  )

  examples = results['documents'][0]
  context = '\n'.join('----\n' + example for example in examples)
  # ------

  messages = [
        {
          "role": "system",
          "content":  "請根據以下書籍資訊，選擇最適合的技能分類。只需要回傳完整分類名稱，不要回傳其他文字，分類只能在以下選一個: 程式語言,Data Science,人工智慧,分散式架構,系統開發,行動軟體開發,資料庫,資訊科學,軟體架構,軟體測試,軟體工程,資訊安全,網站開發,前端開發,架站軟體,網頁設計,Adobe 軟體應用,Office 系列,遊戲開發設計,UI/UX,雲端運算,區塊鏈與金融科技,物聯網 IoT,商業管理類,電子電路電機類,嵌入式系統,視覺影音設計,考試認證,數學,微軟技術,MAC OS 蘋果電腦,其他,兒童專區,製圖軟體應用,語言學習,國家考試,職涯發展,Java,理工類,網路通訊,量子電腦"
        },
        {
            "role": "user",
            "content": context
        },
        {
            "role": "user",
            "content": f"""
              書名: ```{row['title']}```
              描述: ```{str(row['description'])[0:50]}```
              分類:"""
        }
    ]

  response = get_completion(messages, model="gpt-3.5-turbo")
  df2.at[index, 'predict2'] = response

In [29]:
# 計算兩個欄位一致的筆數
correct_predictions2 = (df2['category'] == df2['predict2']).sum()

# 計算準確率
accuracy2 = correct_predictions2 / len(df2)

print(f"準確率: {accuracy2:.2%}")

準確率: 60.61%


## Dynamic few-shot 的補充案例

## 案例

* https://www.microsoft.com/en-us/research/blog/the-power-of-prompting/
* Medprompt+ 還用了 logprobs 參數 來判斷答案的信心指數，推薦 OpenAI 這個 cookbook: https://cookbook.openai.com/examples/using_logprobs
* https://python.langchain.com/docs/modules/model_io/prompts/example_selector_types/similarity
* https://github.com/microsoft/promptbase

## Next

1. 放入向量資料庫的 examples 需要人工好好整理過成為黃金範例
2. 檢索出來的 examples 可再使用 MMR 挑選，以增加多樣性 diverse。因為如果範例出來全部都是同一種分類就比較沒有參考價值了。極端的說，如果向量資料庫裡面有重複的資料，那麼就會撈出重複的資料，透過 MMR 可以排除掉非常相似的內容。

* https://medium.com/tech-that-works/maximal-marginal-relevance-to-rerank-results-in-unsupervised-keyphrase-extraction-22d95015c7c5
* https://python.langchain.com/docs/modules/model_io/prompts/example_selector_types/mmr